In [ ]:
import wandb
import pandas as pd
import matplotlib.pyplot as plt

# 1. Se connecter à l’API
api = wandb.Api()

# 2. Charger le sweep
# Remplace 'entity', 'project' et 'SWEEP_ID' par tes infos
sweep = api.sweep("entity/project/SWEEP_ID")

# 3. Pour chaque run, récupérer l’historique des métriques
all_runs = []
for run in sweep.runs:
    # Choisis les clés qui t’intéressent, ex. 'train_loss', 'val_loss', 'val_accuracy'
    df = run.history(keys=["train_loss", "val_loss", "val_accuracy"], pandas=True)
    df["run_name"] = run.name
    df["step"] = df["_step"]
    all_runs.append(df)

# 4. Concaténer toutes les courbes
df_all = pd.concat(all_runs, ignore_index=True)

# 5. Tracer
plt.figure()
for run_name, group in df_all.groupby("run_name"):
    plt.plot(group["step"], group["val_accuracy"], label=run_name)
plt.xlabel("Step")
plt.ylabel("Validation Accuracy")
plt.legend(bbox_to_anchor=(1.05, 1), loc='upper left')
plt.title("Validation Accuracy par run")
plt.tight_layout()
plt.show()
